In [ ]:
'https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/33538'
'https://www.kaggle.com/frankherfert/local-validation-with-detailed-product-comparison'
    

load the respective training data  
load the respective test data  
add function(s) specific to that dataset  
train each model on the training data  
pickle the trained model (for safe keeping, not totally necessary)  
make predictions on the test data  
pickle the results  

In [1]:
import os

import pandas as pd
import numpy as np

from predict_submit_funcs import submit_fit_score_pred_log



from collections import OrderedDict

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [2]:
os.chdir('../Data/')

# Without adding department

In [3]:
train_df = pd.read_pickle('full_features_train')
test_df = pd.read_pickle('full_features_test')
ord_df = pd.read_csv('orders.csv')
final_order_id = ord_df[ord_df.eval_set=='test'].loc[:,['order_id','user_id']]

## Logistic Regression

In [16]:

def submit_fit_score_pred_log(train_df, test_df, final_order_id):
    """    
    Takes a DataFrame, training, and validation data as its input.
    Returns f1-score, features and their coefficients, and predicted non-re-orders and re-orders.
    """
    test_ids = test_df.loc[:,['product_id','user_id']]
    X_test = test_df.drop(['product_id','user_id'],axis=1) 
    
    X_train = train_df.drop(['product_id','user_id',
                          'latest_cart','in_cart'],axis=1) 
    y_train = train_df['in_cart']
    
    lr = GaussianNB()
    lr.fit(X_train, y_train)
    predictions = pd.DataFrame(lr.predict(X_test))
    
    output = pd.merge(test_ids,predictions,
                      left_index=True,right_index=True)
    
    output = pd.merge(output,final_order_id,
                      on='user_id')
    
    output.columns = ['product_id','user_id',
                      'prediction','order_id']
    
    
    
    return output

In [37]:
predictions = submit_fit_score_pred_log(train_df, test_df, final_order_id)

In [38]:
predictions['prediction'] = predictions.product_id * predictions.prediction
predictions.drop('product_id',axis=1,inplace=True)

In [39]:
def products_concat(series):
    out = ''
    for product in series:
        if product > 0:
            out = out + str(int(product)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [40]:
products_concat(predictions.prediction)

'1 3260 4047 8859 13951 14564 18359 18790 21757 22340 29698 30553 34862 36843 40240 43631 47977 48304 13176 31651 47402 196 5258 8843 11759 13575 37710 49414 1 13176 13575 14444 21572 45051 1 6184 10509 13176 14444 23016 27839 37710 37980 45051 46562 4724 17902 32478 32689 38928 39534 39863 42500 4724 13176 16797 20940 26088 13176 21572 32403 43352 1940 5450 9076 24852 196 23315 46061 4724 5258 11759 14233 21288 8617 10441 11266 11451 21572 22362 26348 31651 32455 37710 39657 44172 45051 46562 40939 46521 1729 13176 21386 6729 43352 196 3298 10258 10441 27839 30591 33198 38928 43154 8013 23909 29487 35102 35383 44799 46749 196 4920 7131 8571 10258 13424 30946 35269 38928 39332 41400 44517 1 196 17924 19660 21100 26348 36472 41400 43721 42450 6184 10509 1774 8859 14228 16419 21115 23909 38672 40897 6184 12341 13176 16797 30633 43352 18583 21573 1 6184 22362 27839 37710 41400 26152 26914 28413 9076 13032 3298 4920 6184 6793 7987 8013 9007 9076 10509 11365 11759 12078 12341 16797 16953 18

In [31]:
predictions.groupby('order_id',as_index=False)['prediction'].apply(lambda x: ' '.join(str(n) for n in x))

0                13107     21463 21709    38777     47766 
1                                                      ...
2           2326     5134                     23794   2...
3                5479      9337          13629         ...
4        1025   4605            13870       24852   271...
5             12779  13198   14077               21903 ...
6                                           35561  40688  
7           2326 4920  8174       15872       21479  22...
8                      9755    14947       20392 20564 ...
9                         12036                        ...
10       365            4589             10132       13...
11        1335      4605 4656   5206  5262 5355 6046   ...
12                                                   47966
13                                  8424             11...
14       1831      17122     27275  31964  36316  39947...
15                                    17820         49328 
16                         8424    9387       13198  13.

In [24]:
predictions = predictions.groupby('order_id',as_index=False)['prediction'].apply(lambda x: ' '.join(str(n) for n in x))

In [25]:
predictions

0        0 0 0 0 13107 0 0 0 0 21463 21709 0 0 0 38777 ...
1        0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2        0 0 0 2326 0 0 0 0 5134 0 0 0 0 0 0 0 0 0 0 0 ...
3        0 0 0 0 0 0 0 0 5479 0 0 0 0 0 9337 0 0 0 0 0 ...
4        1025 0 0 4605 0 0 0 0 0 0 0 0 0 0 0 13870 0 0 ...
5        0 0 0 0 0 12779 0 13198 0 0 14077 0 0 0 0 0 0 ...
6                          0 0 0 0 0 0 0 35561 0 40688 0 0
7        0 0 0 2326 4920 0 8174 0 0 0 0 0 0 15872 0 0 0...
8        0 0 0 0 0 0 0 0 0 0 0 0 0 0 9755 0 0 0 14947 0...
9        0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 12036 0 0 0 ...
10       365 0 0 0 0 0 0 0 0 0 0 0 4589 0 0 0 0 0 0 0 0...
11       0 1335 0 0 0 0 0 4605 4656 0 0 5206 0 5262 535...
12           0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 47966
13       0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
14       1831 0 0 0 0 0 17122 0 0 0 0 27275 0 31964 0 3...
15                 0 0 0 0 0 17820 0 0 0 0 0 0 0 0 49328 0
16       0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 8424 0 0 0.

In [15]:

import re

predictions = predictions.groupby('order_id',as_index=False)['product_id'].apply(lambda x: ' '.join(str(n) for n in x))
predictions = pd.DataFrame(predictions)
final_order_id.reset_index(drop=True,inplace=True)
predictions = pd.merge(final_order_id.order_id,predictions,left_index=True,right_index=True)
predictions.columns = ['order_id','products']
predictions.sort_values('order_id',inplace=True)

In [14]:
predictions.prediction.sum()

132177

In [95]:
predictions.prediction.sum()

571737

In [98]:
predictions.user_id.nunique()

75000

In [92]:
pd.DataFrame.to_csv(predictions,'my_order_predictions.csv',index=False)

In [49]:
test.sort_values('order_id')

,order_id,products
29949,8188,1706 7682 8519 8887 9623 11278 13440 13954 141...
26166,8363,539 649 714 2838 7301 10403 12140 12638 12651 ...
60348,10955,2077 2132 2664 2809 3773 4116 4227 4341 4932 5...
46959,12187,12341 15258 22179 25265 31380
21475,14817,1 130 500 940 1689 2691 3399 4913 11759 12341 ...
61865,16160,1090 1212 3211 3729 4428 4605 4853 5240 5284 5...
42048,17751,2245 4097 13176 13252 15424 17426 19294 20024 ...
20666,18113,116 196 651 889 1747 2433 3298 4724 4938 5258 ...
31693,18531,1185 1337 1909 1947 2298 3843 4939 5042 8043 9...
42595,18542,45 204 260 1202 3952 3988 4605 4614 5212 5450 ...


## Gaussian Naive Bayes

## Random Forest

# With Polynomial Features

In [ ]:
df = pd.read_pickle('poly_feats_without_dep_df.pkl')

## Gaussian Naive Bayes

# Adding Departments

In [ ]:
df = pd.read_pickle('full_features_with_dep')

## Gaussian Naive Bayes

## Multinomial Naive Bayes

# After Scaling Data

In [ ]:
df = pd.read_pickle('scaled_feats_df')

## Logistic Regression

## Gaussian Naive Bayes

# Scaled Polynomial Features

In [ ]:
df = pd.read_pickle('scaled_poly_feats_without_dep_df.pkl')

## Logistic Regression

## Gaussian Naive Bayes